In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, to_timestamp, date_format, coalesce, median, lit
)
from pyspark.sql.types import DoubleType, StringType

# =======================================================================
# 1. CONFIGURAÇÃO E INICIALIZAÇÃO DO SPARK
# =======================================================================
conf = SparkConf()
# Pacotes necessários para a comunicação S3A (Hadoop AWS e AWS SDK)
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.11.901')
# Provedor de credenciais (assumindo Instance Profile)
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')

# CORREÇÃO DO ERRO DE PARSING DE DATA/HORA (SparkUpgradeException)
conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

# Inicializa a sessão Spark com a configuração S3 e a correção de data/hora
spark = SparkSession.builder.appName("DataCleaningClimaTempo").config(conf=conf).getOrCreate()

# Definição dos caminhos S3
INPUT_PATH_WEATHER = "s3a://bucket-raw-upa-connect-sofh/basesExternas/ClimaTempo/output.csv"
INPUT_PATH_AIR = "s3a://bucket-raw-upa-connect-sofh/basesExternas/ClimaTempo/outpu-air.csv"

# =======================================================================
# 2. DEFINIÇÕES DE CAMINHO PARA SALVAMENTO
# =======================================================================
# Define o diretório de destino final
FINAL_OUTPUT_DIR = "s3a://bucket-trusted-upa-connect-sofh/BasesExternas/ClimaTempo"

# Define o nome do arquivo final (conforme solicitado)
FINAL_FILENAME = "ClimaTempoTratada.csv"

# Define o caminho temporário (subdiretório dentro da pasta principal)
TEMP_STAGING_DIR = f"{FINAL_OUTPUT_DIR}/_temp_staging"


# =======================================================================
# 3. LEITURA DOS DADOS DO S3
# =======================================================================
print("Lendo dados do S3...")

df_weather = spark.read.option("header", "true").csv(INPUT_PATH_WEATHER)
df_air = spark.read.option("header", "true").csv(INPUT_PATH_AIR)

# =======================================================================
# 4. PRÉ-PROCESSAMENTO: CONVERSÃO DE TIPOS E JOIN
# =======================================================================

air_cols_to_cast = ["pm10", "pm2_5", "carbon_monoxide", "nitrogen_dioxide", "ozone", "sulphur_dioxide"]

for c in air_cols_to_cast:
    df_air = df_air.withColumn(c, col(c).cast(DoubleType()))

df_final = df_weather.join(df_air, on="date", how="outer")
print("\nDataFrame após o merge:")
df_final.printSchema()


# =======================================================================
# 5. TRATAMENTO DE VALORES AUSENTES (MEDIANA)
# =======================================================================

print("\nCalculando medianas para preenchimento de NaNs...")
median_values = df_final.agg(*(median(col(c)).alias(f"median_{c}") for c in air_cols_to_cast)).collect()[0]

fill_map = {c: median_values[f"median_{c}"] for c in air_cols_to_cast}

for col_name, med_value in fill_map.items():
    df_final = df_final.withColumn(
        col_name,
        coalesce(col(col_name), lit(med_value).cast(DoubleType()))
    )

print("Valores nulos preenchidos com a mediana.")


# =======================================================================
# 6. TRATAMENTO E FORMATAÇÃO DA COLUNA 'DATE'
# =======================================================================

df_final = df_final.withColumn(
    "date_converted",
    to_timestamp(col("date"), "yyyy-MM-dd HH:mm:ss").cast(StringType())
)

df_final = df_final.withColumn(
    "DATA_ISO",
    date_format(col("date_converted"), "yyyy-MM-dd'T'HH:mm:ss")
).drop("date_converted", "date")


# =======================================================================
# 7. LIMPEZA FINAL
# =======================================================================

for c in df_final.columns:
    if c not in ["DATA_ISO"]:
        try:
            df_final = df_final.withColumn(c, col(c).cast(DoubleType()).cast("decimal(10, 2)"))
        except:
            pass

df_final = df_final.toDF(*(c.upper() for c in df_final.columns))

final_cols = [c for c in df_final.columns if c != "DATA_ISO"] + ["DATA_ISO"]
df_final = df_final.select(*final_cols)


# =======================================================================
# 8. SALVANDO E RENOMEANDO O RESULTADO NO S3
# =======================================================================

# 1. Escreve o resultado no caminho temporário
print(f"\nEscrevendo dados temporariamente em: {TEMP_STAGING_DIR}")
# Note: Removi .coalesce(1) para evitar potencial erro de memória, 
# mas se for crítico ter um único arquivo, pode ser re-adicionado
df_final.coalesce(1).write \
    .option("header", "true") \
    .option("delimiter", ";") \
    .mode("overwrite") \
    .csv(TEMP_STAGING_DIR)

# 2. Renomeia o arquivo gerado
try:
    # Acessa a classe 'Path' da JVM através do gateway do Spark
    Path = spark._jvm.org.apache.hadoop.fs.Path
    
    # Acessa a configuração do Hadoop
    hadoop_conf = spark._jsc.hadoopConfiguration()
    
    # Obtém o objeto FileSystem para o caminho temporário
    fs = Path(TEMP_STAGING_DIR).getFileSystem(hadoop_conf)

    # Encontra o arquivo gerado (part-00000-*.csv) dentro do diretório temporário
    list_status = fs.globStatus(Path(TEMP_STAGING_DIR + "/part-00000-*.csv"))

    if list_status:
        # Pega o caminho completo do arquivo gerado
        generated_file_path = list_status[0].getPath()

        # Define o caminho final e o nome específico para o arquivo
        final_output_path = Path(f"{FINAL_OUTPUT_DIR}/{FINAL_FILENAME}")

        # Renomeia (move) o arquivo para o caminho e nome definitivos
        fs.rename(generated_file_path, final_output_path)
        
        # 3. Deleta o diretório temporário (que ficou vazio) e outros arquivos de metadados
        fs.delete(Path(TEMP_STAGING_DIR), True) 
        
        print(f"\n✅ Dados salvos e renomeados com sucesso para: {final_output_path}")

    else:
        print("\nErro: Não foi possível encontrar o arquivo CSV gerado (part-00000-*.csv) no caminho temporário.")

except Exception as e:
    print(f"\nOcorreu um erro durante a renomeação do arquivo no S3: {e}")

# Encerra a sessão Spark
spark.stop()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-205ceb27-a542-4548-bdc8-4655aa196caa;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 314ms :: artifacts dl 37ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	:: evicted modules:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 by [com.amazonaws#aws-java-sdk-bundle;1.12.262] in [default]
	---------------------------------------------------------------------
	|     

Lendo dados do S3...


25/10/16 02:26:05 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties



DataFrame após o merge:
root
 |-- date: string (nullable = true)
 |-- temperature_2m: string (nullable = true)
 |-- relative_humidity_2m: string (nullable = true)
 |-- rain: string (nullable = true)
 |-- precipitation: string (nullable = true)
 |-- pm10: double (nullable = true)
 |-- pm2_5: double (nullable = true)
 |-- carbon_monoxide: double (nullable = true)
 |-- nitrogen_dioxide: double (nullable = true)
 |-- ozone: double (nullable = true)
 |-- sulphur_dioxide: double (nullable = true)


Calculando medianas para preenchimento de NaNs...


Valores nulos preenchidos com a mediana.

Escrevendo dados temporariamente em: s3a://bucket-trusted-upa-connect-sofh/BasesExternas/ClimaTempo/_temp_staging


25/10/16 02:26:22 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
25/10/16 02:26:22 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.



✅ Dados salvos e renomeados com sucesso para: s3a://bucket-trusted-upa-connect-sofh/BasesExternas/ClimaTempo/ClimaTempoTratada.csv
